In [3]:
!pip3 install -q torch openai 

You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.7/bin/python3.7 -m pip install --upgrade pip' command.


In [7]:
!pip3 install -q langchain nbformat 

You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.7/bin/python3.7 -m pip install --upgrade pip' command.


In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage
from langchain.prompts import (
    ChatPromptTemplate,
    FewShotChatMessagePromptTemplate,
)

import nbformat
import json
import os

## Reading data

In [30]:
def read_notebook(notebook_path):
        with open(notebook_path, 'r', encoding='utf-8') as f:
            return nbformat.read(f, as_version=4)

def extract_cells(notebook):
    cells = []
    for cell in notebook.cells:
        if cell.cell_type in ['markdown', 'code']:
            cells.append({
                'type': cell.cell_type,
                'content': cell.source
            })
    return cells

def format_for_gpt(cells):
    formatted_data = {
        "cells": cells
    }
    return json.dumps(formatted_data, indent=4)

In [51]:
notebook = read_notebook("/Users/shobanasiranjeevilu/Downloads/sample_work/sample_work/sample_assignments/assignment_4.ipynb")
extracted_cells = extract_cells(notebook)
formatted_data = format_for_gpt(extracted_cells)

In [115]:
formatted_data

'{\n    "cells": [\n        {\n            "type": "code",\n            "content": "import pandas as pd"\n        },\n        {\n            "type": "code",\n            "content": "df=pd.read_csv(\\"yelp_2.csv\\")\\ndf"\n        },\n        {\n            "type": "markdown",\n            "content": "Average number of words in each review (define \\u201cwords\\u201d however you like but be explicit about it)"\n        },\n        {\n            "type": "code",\n            "content": "import pandas as pd\\n\\n\\ndata = {\\n    \'Unnamed: 0\': [0, 1, 2, 3, 4],\\n    \'text\': [\\n        \\"My wife took me here on my birthday for breakfast\\",\\n        \\"I have no idea why some people give bad reviews\\",\\n        \\"Love the gyro plate. Rice is so good and I also...\\",\\n        \\"Rosie, Dakota, and I LOVE Chaparral Dog Park!!...\\",\\n        \\"General Manager Scott Petello is a good egg!!!...\\"\\n    ]\\n}\\n\\n\\ndf = pd.DataFrame(data)\\n\\n\\ndef count_words(text):\\n    wo

## Model creation

In [12]:
chat = ChatOpenAI(temperature=0, model_name="gpt-4", openai_api_key="")


## Simple prompting

In [83]:
messages = [
    SystemMessage(
        content=""" You are a helpful assistant that evaluate python program and grad from 1-100 
        and explain why you gave such grade. make sure to provide your response in bullet points"""
    ),
    HumanMessage(
        content=  formatted_data )
]
response = chat(messages)

In [84]:
import re
overall_grade = re.search(r"(\d+) out of 100", response.content)
points = re.findall(r"- (.+?)\s+\(\+?(-?\d+) points\)", response.content)
print(f"Overall Grade: {overall_grade.group(1)} out of 100")
for point, grade_change in points:
    print(f"Feedback: {point}\nGrade points: {grade_change} points")

Overall Grade: 90 out of 100
Feedback: The code is well-structured and organized into different cells, each performing a specific task. This makes the code easy to read and understand.
Grade points: 20 points
Feedback: The code uses pandas, a powerful data analysis library in Python, to perform operations on the data. This is a good choice for handling data in Python.
Grade points: 20 points
Feedback: The code correctly calculates the average number of words in each review, the count of reviews by year-month, and the average rating of any review marked as "cool". The logic used in these calculations is correct.
Grade points: 30 points
Feedback: The code uses the `apply` function to apply a function to each element in a column of the DataFrame. This is a good use of pandas functionality.
Grade points: 10 points
Feedback: The code uses the `groupby` function to group the data by year-month and count the number of reviews in each group. This is a good use of pandas functionality.
Grade po

## Instruction prompting

In [85]:
messages = [
    SystemMessage(
        content=""" You are a grading bot that will help a teaching assistant grade student submissions for a programming task. Please follow the
                instructions below to grade the student submission. In your assessment, you must always fill in the following bullet points:
                Grade: 0-100
                Summarised feedback: A short one or two sentence summary of your assessment.
                Possible errors: Write any errors that you might find in a bullet point list. If you find noerrors, write "None". If there are several,
                which there very well might be, please list them all.
                
                You shall under no circumstanses comment on unnecessary improvements, only those who are relevant to the task. Only comment on mistakes
                that makes the code fail the requirements of the task. Elements such as effiency is not relevant if its not explicitly a requirement
                of the task."""
    ),
    HumanMessage(
        content=  formatted_data )
]

In [86]:
response = chat(messages)

In [87]:
grade = re.search(r"Grade: (\d+)", response.content)
summarised_feedback = re.search(r"Summarised feedback: (.+)", response.content)
possible_errors = re.search(r"Possible errors: (.+)", response.content)

# Displaying the results
grade = grade.group(1) if grade else "Unknown"
summarised_feedback = summarised_feedback.group(1) if summarised_feedback else "No feedback provided"
possible_errors = possible_errors.group(1) if possible_errors else "No errors listed"

print(f"Grade: {grade}")
print(f"Summarised Feedback: {summarised_feedback}")
print(f"Possible Errors: {possible_errors}")

Grade: 100
Summarised Feedback: The student has successfully completed all the tasks as per the requirements. The code is correct and produces the expected output.
Possible Errors: None


## Few shot prompting

In [6]:
#Example 1
notebook = read_notebook("/Users/shobanasiranjeevilu/Downloads/sample_work/sample_work/sample_assignments/assignment_3.ipynb")
extracted_cells = extract_cells(notebook)
example_1 = format_for_gpt(extracted_cells)

#Example 2

notebook = read_notebook("/Users/shobanasiranjeevilu/Downloads/sample_work/sample_work/sample_assignments/assignment_2.ipynb")
extracted_cells = extract_cells(notebook)
example_2 = format_for_gpt(extracted_cells)

#Example 3

notebook = read_notebook("/Users/shobanasiranjeevilu/Downloads/sample_work/sample_work/sample_assignments/assignment_7.ipynb")
extracted_cells = extract_cells(notebook)
example_3 = format_for_gpt(extracted_cells)

In [7]:
examples = [
    {"input": example_1, "output": "Grade:85, summarized Feedback: Question2 is not executable, Question3 gives no output. Add notebook with executed output/ output files next time, Possible Errors: None" },
    {"input": example_2, "output": "Grade:85, summarized Feedback: Question 1 will not execute, Possible Errors: Incorrect class name is used while invoking"},
    {'input': example_3, "output": "Grade:20, summarized Feedback: No map/reduce logic has been written to do the task. DataFrames are not allowed"}
]

In [10]:
example_prompt = ChatPromptTemplate.from_messages(
    [
        ("user", "{input}"),
        ("ai", "{output}"),
    ]
)

few_shot_prompt = FewShotChatMessagePromptTemplate(
    
    input_variables=["input"],
    examples=examples,
    example_prompt= example_prompt
    )

In [136]:
print(few_shot_prompt.format())

Human: {
    "cells": [
        {
            "type": "code",
            "content": "!pip install MRJob\n!pip install python-dateutil"
        },
        {
            "type": "code",
            "content": "from google.colab import drive\ndrive.mount('/content/drive')\nimport pandas as pd\ndata = pd.read_csv('/content/drive/MyDrive/bigdata/yelp.csv')"
        },
        {
            "type": "code",
            "content": "%%file average_words_per_review.py\nfrom mrjob.job import MRJob\nclass MRAverageWordsPerReview(MRJob):\n    def mapper(self, _, line):\n        # Assuming each line is a review.\n        words = line.split()\n        yield \"review\", len(words)\n    def reducer(self, key, values):\n        total_words = 0\n        total_reviews = 0\n        for value in values:\n            total_words += value\n            total_reviews += 1\n        average_words = total_words / total_reviews\n        yield key, average_words\nif __name__ == '__main__':\n    MRAverageWordsPerRev

### Few shot without context(Rubic)

In [138]:
final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant that evaluate python program and grad from 1-100 and \
         explain why you gave such grade. make sure to provide your response in bullet points"),
        few_shot_prompt,
        ("user", "{input}"),
    ]
)
chain = final_prompt | chat
response_without_context = chain.invoke({"input": formatted_data})
points = response_without_context.content.split("\n")
for point in points:
    print(f"{point}")

Grade: 70

Feedback:

- The code is well-structured and easy to read.
- The use of pandas for data manipulation is appropriate and efficient.
- The logic used to solve the problems is correct and the code produces the expected results.
- However, the code is not scalable. It works well for small datasets but for larger datasets, it would be more efficient to use a map-reduce framework like Hadoop or Spark.
- The code could be improved by adding more comments to explain the logic and the steps.
- The code could also be improved by defining functions for repeated code blocks.
- The code does not handle exceptions or errors. It would be better to add error handling to make the code more robust.
- The code does not have any unit tests. It would be better to add tests to ensure the code works as expected.


### few shot with context

In [53]:
final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant that evaluate python program and grad from 1-100 and explain why you gave such grade. make sure to provide your response in bullet points\
         Write three Python scripts using mrjob that tell me:\
Average number of words in each review (define “words” however you like but be explicit about it)\
Count of reviews by year-month (eg “2021-09”)\
Average rating of any review marked ”cool” (eg where cool != 0)"
),
        few_shot_prompt,
        ("user", "{input}"),
    ]
)
chain = final_prompt | chat
response_without_context = chain.invoke({"input": formatted_data})
points = response_without_context.content.split("\n")
for point in points:
    print(f"{point}")

Grade: 20, summarized Feedback: No map/reduce logic has been written to do the task. DataFrames are not allowed


        So, I have inferenced a gpt-4 model to identify best prompting way to get the desired results on Evaluating student assignments. 

        Hence performing few-shot prompting has showed significant improvement but still the model focused more coding standards.

        To improve further, provided content on what the model shoud be looking for in grading the assignments. With that I was able to achieve the desired output. 

        Using this techinique going forward.

## Inference on GPT  (few-shot prompting with context)

In [14]:
final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant that evaluate programming assignments and check all the instructions are followed and grade it from 1-100 and provide Feedback what went wrong.Provide Reasoning on why you gave such grade. \
            Given the dataset about weather in Australia , predict rain using models. \
         - must have used ML model to perform prediction. \
          - Include VectorAssembler successfully to perform feature transformation  "
),
        
        few_shot_prompt,
        ("user", "{input}"),
    ]
)
chain = final_prompt | chat


In [28]:
folder_path = '/Users/shobanasiranjeevilu/Downloads/sample_work/sample_work/inference'
file_names,files = [],[]
# Collecting .ipynb files and avoiding hidden/system files
for file in os.listdir(folder_path):
    full_path = os.path.join(folder_path, file)
    if os.path.isfile(full_path) and file.endswith('.ipynb') and not file.startswith('.'):
        file_names.append(full_path)
        

#### To avoid RateLimitException taking 3 assignments per loop

In [34]:
output = []
for file in file_names[:3]:
    notebook = read_notebook(file)
    extracted_cells = extract_cells(notebook)
    example = format_for_gpt(extracted_cells)
    response_with_context = chain.invoke({"input": example})
    output.append(response_with_context)

In [36]:
for file in file_names[3:6]:
    notebook = read_notebook(file)
    extracted_cells = extract_cells(notebook)
    example = format_for_gpt(extracted_cells)
    response_with_context = chain.invoke({"input": example})
    output.append(response_with_context)

In [38]:
for file in file_names[6:9]:
    notebook = read_notebook(file)
    extracted_cells = extract_cells(notebook)
    example = format_for_gpt(extracted_cells)
    response_with_context = chain.invoke({"input": example})
    output.append(response_with_context)

In [42]:
for file in file_names[9:]:
    notebook = read_notebook(file)
    extracted_cells = extract_cells(notebook)
    example = format_for_gpt(extracted_cells)
    response_with_context = chain.invoke({"input": example})
    output.append(response_with_context)

In [45]:
file_names

['/Users/shobanasiranjeevilu/Downloads/sample_work/sample_work/inference/assignment_4.ipynb',
 '/Users/shobanasiranjeevilu/Downloads/sample_work/sample_work/inference/assignment_6.ipynb',
 '/Users/shobanasiranjeevilu/Downloads/sample_work/sample_work/inference/assignment_10.ipynb',
 '/Users/shobanasiranjeevilu/Downloads/sample_work/sample_work/inference/assignment_1.ipynb',
 '/Users/shobanasiranjeevilu/Downloads/sample_work/sample_work/inference/assignment_3.ipynb',
 '/Users/shobanasiranjeevilu/Downloads/sample_work/sample_work/inference/assignment_7.ipynb',
 '/Users/shobanasiranjeevilu/Downloads/sample_work/sample_work/inference/assignmebt_2.ipynb',
 '/Users/shobanasiranjeevilu/Downloads/sample_work/sample_work/inference/assignment_5.ipynb',
 '/Users/shobanasiranjeevilu/Downloads/sample_work/sample_work/inference/assignment_9.ipynb',
 '/Users/shobanasiranjeevilu/Downloads/sample_work/sample_work/inference/assignment_8.ipynb']

In [43]:
output

[AIMessage(content="Grade: 95\n\nThe student has done a good job in following the instructions. They have used a machine learning model (Random Forest) to predict whether it will rain or not. They have also used VectorAssembler for feature transformation. The code is well-structured and the logic is correct. \n\nHowever, the student has not provided any comments or explanations about the steps they are performing, which makes it a bit difficult to follow the logic. It would be better if they could provide some explanations or comments in their code. \n\nThe student could also improve their work by performing some exploratory data analysis before building the model. This would give them a better understanding of the data and could potentially improve the model's performance. \n\nFinally, the student could try using different machine learning models and compare their performance. This would give them a better understanding of which model works best for this particular problem."),
 AIMess

In [50]:
with open('gpt_4_inference.jsonl', 'w') as file:
    for i,evaluation in enumerate(output):
        json_line = json.dumps(evaluation.content)  # Converting dictionary to JSON string
        file.write(json_line + '\n') 



In [47]:
for i,evaluation in enumerate(output):
    feedback = evaluation.content.split("\n")

    print(file_names[i])

    for points in feedback:
        print(points)

/Users/shobanasiranjeevilu/Downloads/sample_work/sample_work/inference/assignment_4.ipynb
Grade: 95

The student has done a good job in following the instructions. They have used a machine learning model (Random Forest) to predict whether it will rain or not. They have also used VectorAssembler for feature transformation. The code is well-structured and the logic is correct. 

However, the student has not provided any comments or explanations about the steps they are performing, which makes it a bit difficult to follow the logic. It would be better if they could provide some explanations or comments in their code. 

The student could also improve their work by performing some exploratory data analysis before building the model. This would give them a better understanding of the data and could potentially improve the model's performance. 

Finally, the student could try using different machine learning models and compare their performance. This would give them a better understanding of 